In [ ]:
# | default_exp cash

In [ ]:
from budgeting_tools.ynab_utils import get_budgeted_balance, get_account_balances
import os

In [ ]:
balances = get_account_balances()
checking_balances = balances.query('type == "checking"')
creditcard_balances = balances.query('type == "creditCard"')
savings_balances = balances.query('type == "savings"')
other_balances = balances.query('type not in ["checking", "creditCard", "savings"]')

In [ ]:
budgeted_balance = get_budgeted_balance(os.environ["YNAB_TOKEN"])

In [ ]:
budgeted_balance - checking_balances["balance"].sum() / 1000

-963.9400000000005

In [ ]:
import datetime
import requests
import pandas as pd
import json


def calc_savings_transfer(token):
    saving_groups = [
        "394b0df3-9457-4a5d-913c-72ba3868dc64",
        "db3dbf9f-91c8-4ec8-9b49-847570e5e5dc",
    ]
    date = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m") + "-01"
    month = requests.get(
        f"https://api.youneedabudget.com/v1/budgets/last-used/months/{date}?access_token={token}"
    )
    income = json.loads(month.content)["data"]["month"]["to_be_budgeted"] / 1000
    category_balances = pd.DataFrame(
        json.loads(month.content)["data"]["month"]["categories"]
    )
    category_balances["balance"] = category_balances["balance"].apply(
        lambda x: x / 1000
    )
    category_balances = category_balances[category_balances.balance > 0]
    category_balances = category_balances[
        category_balances.name != "Inflow: Ready to Assign"
    ]
    cash_accounts = category_balances[
        ~category_balances.category_group_id.isin(saving_groups)
    ]
    required_cash = income + cash_accounts.balance.sum()
    accounts = requests.get(
        f"https://api.youneedabudget.com/v1/budgets/last-used/accounts?access_token={token}"
    )
    accounts = pd.DataFrame(json.loads(accounts.content)["data"]["accounts"])
    checking_balance = accounts.query("name == 'MACU Checking'").balance[0] / 1000
    transfer = checking_balance - required_cash
    if transfer > 0:
        print(f"You should transfer ${transfer} from MACU Checking to Savings")
    else:
        print(f"You should transfer ${transfer * -1} from Savings to MACU Checking")
    return transfer, required_cash

In [ ]:
calc_savings_transfer(os.environ["YNAB_TOKEN"])

You should transfer $963.9400000000005 from MACU Checking to Savings


(963.9400000000005, 14492.32)